# Plot the AMR solution along a transect

This Jupyter notebook is available in `$GTT/CopalisBeach/exercise1/plot_transect.ipynb` within the [GeoClaw Tsunami Tutorial](https://rjleveque.github.io/geoclaw_tsunami_tutorial).

This notebook shows how to plot a single time frame of the AMR solution and how to extract a one-dimensional transect of the solution from the AMR structure.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.visclaw import gridtools
from clawpack.pyclaw import Solution
import setplot 

In [ ]:
if 1:
    # use this to fetch sample_results from the online data repository:
    if not os.path.isdir('sample_results'):
        import fetch_sample_results  # fetches directory
    outdir = 'sample_results/_output'
else:
    # use this if you have run the code locally to create '_output'
    outdir = '_output'

## Use setplot.py to specify how to plot a time frame as a 2D plot

In [ ]:
plotdata = setplot.setplot()
plotdata.outdir = outdir
plotdata.print_fignos = [1] # which figno to show
plotdata.printfigs = False  # do not make png file, show here

### plot a single frame:

In [ ]:
plotdata.plotframe(frameno=6)

## Define a 1-dimensional transect

In [ ]:
x1 = -124.24;  y1 = 47.14
x2 = -124.15; y2 = 47.11
npts = 1000
xtrans = linspace(x1,x2,npts)
ytrans = linspace(y1,y2,npts)

## A function to plot one frame together with solution on a transect

The function `clawpack.visclaw.gridtools.grid_output_2d` can be used to extract values along a one-dimensional transect from a 2D AMR solution.  First we read in the full AMR structure and solution using `clawpack.pyclaw.Solution`

In [ ]:
def make_plots(frameno):

    # read in the AMR solution for this time frame:
    framesoln = Solution(frameno, path=outdir, file_format='binary')

    # Plot the 2D solution:
    plotdata.plotframe(frameno)
    
    # add a line showing the transect location:
    plot(xtrans, ytrans, 'k')
    text(x1+0.002, y1-0.002, 'Transect', rotation=-17)
    
    for method in ['linear', 'nearest']:
        # extract solution along the transect (at discrete points xtrans,ytrans):
        h_trans = gridtools.grid_output_2d(framesoln, 0, xtrans, ytrans, method=method)
        eta_trans = gridtools.grid_output_2d(framesoln, -1, xtrans, ytrans, method=method)
        B_trans = eta_trans - h_trans  # topo reconstructed from eta - h
        
        figure(figsize=(10,4))
        plot(xtrans,eta_trans,'b')  # plot water surface blue
        plot(xtrans,B_trans,'g')    # plot topo green (on top of eta where h=0)
        ticklabel_format(useOffset=False)
        grid(True)

### Apply the function at several times

Note in the plots below that the same transect data is plotted in two ways: 

1. using piecewise bilinear interpolation between finite volume solution values on the various grids
2. with zero-order interpolation, i.e. the value at each of the points on the transect is simply the value in the finite volume grid cell in which the point lies.

The first method is often preferable since it give smoother looking results, but the second approach better shows the underlying grid resolution of the simulation.

It also shows that at later times the computational grid is more refined than at early times (since the adaptive mesh refinement adds more levels after the tsunami arrives).

In [ ]:
make_plots(frameno=0)

In [ ]:
make_plots(frameno=1)

Note in the plots above that the water surface is still at time 10 minutes, but the co-seismic subsidence has dropped both the land and water relative to time 0.

Jumping ahead to 40 minutes, we see the main tsunami arriving:

In [ ]:
make_plots(frameno=4)

In [ ]:
make_plots(frameno=5)

Note that the piecewise constant solution seems to show unequal cell sizes and the topography appears to have some very narrow spikes that do not show up in the linear version. That is because the transect chosen is at an angle to the grid and cuts through the middle of some cells but only nicks a corner of other cells.  Since 1000 points are used along the transect, it has much higher resolution than the underlying computational cells, and so there may be one or transect points in a cell even if the transect barely touches it.  Bilinear interpolation between the 4 neighboring cell centers gives a much smoother interpolant.